In [ ]:
from pytorch_model import load_wpod
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision import transforms, utils, models
from torch import nn
import torch
from src.label import Label, Shape
from src.utils import getWH, nms, im2single, IOU_centre_and_dims
from src.projection_utils import getRectPts, find_T_matrix
import time
from utils import *
from tqdm import tqdm

In [ ]:
wpod_net = load_wpod()
wpod_net.to(device)
wpod_net.eval()
lp_threshold = .5

In [ ]:
def imstat(im):
    Ivehicle = cv2.imread(im)
    plt.imshow(Image.open(im))
    plt.show()
    ratio = float(max(Ivehicle.shape[:2]))/min(Ivehicle.shape[:2])
    side  = int(ratio*288.)
    max_dim = min(side + (side%(2**4)),608)
    I = im2single(Ivehicle)
    net_step = 2**4
    min_dim_img = min(I.shape[:2])
    factor 		= float(max_dim)/min_dim_img

    w,h = (np.array(I.shape[1::-1],dtype=float)*factor).astype(int).tolist()
    w += (w%net_step!=0)*(net_step - w%net_step)
    h += (h%net_step!=0)*(net_step - h%net_step)
    Iresized = cv2.resize(I,(w,h))

    Tn = Iresized.copy()
    Tn = Tn.reshape((1,Tn.shape[0],Tn.shape[1],Tn.shape[2]))
    Tn = torch.tensor(Tn, device=device).permute(0,3,1,2)
    Yr = wpod_net(Tn).permute(0,2,3,1)[0].cpu().detach().numpy()
    Probs = Yr[...,0]
    acc = np.amax(Probs)
    print("Acc is {}%\n".format(round(100*acc,4)))
    return acc

In [ ]:
#Test Physical. Don't run if you don't have the test images
root = "lp200/lp200_"
names = ["stripefull"]
for i in range(64):
    names.append(root + "{}".format(i))
accs = []
dodge = 0
total = -1
for name in names:
    print(name)  
    im = name + ".jpg"
    acc = imstat(im)
    accs.append(acc)
    if acc < lp_threshold:
        dodge += 1
    total += 1
    
#remove full from stats
accs.pop(0)

print("\n\nAvg Acc: {}%\n".format(round(100*torch.tensor(accs).mean().item(),4)))

print("dodge_success: {:.6}%".format(100 * dodge / total))


In [ ]:
cop = accs
plt.plot(accs)
plt.show()
cop.sort()
plt.plot(cop)
plt.show()

In [ ]:
accs.dtype